<a href="https://colab.research.google.com/github/danielgarciabr/agenteIAexperimental/blob/main/Agente_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber sobre as próximas edições é:

*   **Acessar a página da Imersão:** Procure no site da Alura por "Imersão IA com Google Gemini" ou "Google Gemini Alura". Lá, você pode encontrar informações sobre edições passadas e, possivelmente, um formulário para se inscrever e receber notificações sobre futuras turmas.
*   **Seguir a Alura nas redes sociais:** A Alura costuma divulgar seus eventos e cursos em plataformas como LinkedIn, Instagram, Twitter e Facebook.
*   **Inscrever-se na newsletter da Alura:** Assim, você receberá novidades e informações sobre os próximos cursos e imersões diretamente no seu e-mail.

In [6]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A última Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição se encerraram no dia 11 de maio de 2025.

Para saber sobre as próximas edições, você pode acompanhar o site da Alura: [https://www.alura.com.br/imersao-ia-google-gemini-ii](https://www.alura.com.br/imersao-ia-google-gemini-ii).


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: alura.com.br, starten.tech



In [8]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [21]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search],
        instruction="""
        Você é um assitente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as ultimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        foque em no máximos 5 lançamentos relevantes, com base na quantidade e entusiasmo ds notícias sobre ele.
        se o um tema tiver poucas notícias ou reações entusiamadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        esses lançamento relevante devem ser atuais, de no máximo um mês antes da data de hoje
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


In [13]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar ferramentas de busca do google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um poste sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com na base nas suas pesquisas,
        e retornar esse temas, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [29]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [30]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [31]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print("Maravilha! Vamos então criar o post sobre as novidadades em ", topico)
  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultados do agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("---------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📝 Resultados do agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("---------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📝 Resultados do agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("---------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📝 Resultados do agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("---------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agente de ia


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-70' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

Maravilha! Vamos então criar o post sobre as novidadades em  agente de ia

--- 📝 Resultados do agente 1 (Buscador) ---



> Para encontrar os lançamentos de agentes de IA mais relevantes e recentes, farei algumas pesquisas no Google. Tentarei identificar os lançamentos que geraram mais notícias e entusiasmo nos últimos meses.
> 
> 
> Com base nas minhas pesquisas, aqui estão alguns dos lançamentos e tendências mais relevantes no espaço de agentes de IA em maio de 2025:
> 
> 1.  **IBM watsonx Orchestrate:** A IBM anunciou atualizações para o watsonx Orchestrate, oferecendo soluções para integrar, personalizar e implementar agentes de IA. Inclui agentes pré-construídos para RH, vendas e compras, com integração com mais de 80 aplicações empresariais.
> 2.  **Agentes de IA no e-commerce:** A Mastercard, em parceria com a Microsoft e IBM, lançou o Mastercard Agent Pay, permitindo que agentes de IA processem compras e realizem pagamentos em nome dos usuários. O ChatGPT também lançou uma funcionalidade para encontrar e comprar produtos diretamente na plataforma.
> 3.  **Tendências de Agentes de IA segundo a Microsoft:** A Microsoft destaca que em 2025, uma nova geração de agentes de IA poderá lidar com várias tarefas, inclusive de forma autônoma, transformando cada processo de negócios.
> 4.  **Crescimento do mercado de agentes de IA:** Previsões indicam que o mercado de agentes de IA será avaliado em US$ 47,1 bilhões até 2025, com um crescimento anual de 44,8% até 2030.
> 5.  **Agentes de IA em Criptomoedas:** Há um crescente interesse em agentes de IA no espaço de criptomoedas, com foco em negociação, automação de tarefas e gerenciamento de comunidades descentralizadas. Alguns exemplos incluem AIXBT da Virtuals, AI16z, e Freysa IA.
> 
> É importante notar que muitas empresas estão investindo fortemente no aprimoramento desses sistemas, tornando-os mais personalizados e eficientes. No entanto, desafios como privacidade de dados, regulação e impacto no mercado de trabalho exigem um debate amplo para garantir que esse avanço tecnológico seja inclusivo e sustentável.
> 


---------------------------------------------

--- 📝 Resultados do agente 2 (Planejador) ---



...

---------------------------------------------

--- 📝 Resultados do agente 3 (Redator) ---



> 🤖 O Futuro Chegou: Agentes de IA Autônomos Transformando o Mundo dos Negócios! 🚀
> 
> Já imaginou um mundo onde cada processo da sua empresa é otimizado por agentes de IA que trabalham sozinhos? 🤯 A Microsoft acredita que esse futuro está mais perto do que pensamos!
> 
> ✨ Agentes Multifuncionais:
> Esses novos agentes são tipo "faz tudo" da tecnologia, integrando diversas habilidades e conhecimentos para automatizar tarefas como:
> 🗓️ Agendamento inteligente
> 📊 Análise de dados complexa
> 💬 Atendimento ao cliente personalizado
> 
> 💼 Transformação nos Negócios:
> Na saúde, eles monitoram pacientes e agendam consultas. Nas finanças, analisam o mercado e detectam fraudes. Na logística, otimizam rotas e gerenciam estoques! 🩺💰🚚
> 
> 🗣️ A Microsoft Acredita:
> A gigante da tecnologia está investindo pesado nessa revolução, e nós da Alura estamos de olho em cada novidade! 👀
> 
> 🤔 E você, como acha que os agentes de IA autônomos podem transformar a sua área de atuação? Compartilhe suas ideias nos comentários! 👇
> 
> \#IA \#AgentesDeIA \#InteligenciaArtificial \#Microsoft
> 


---------------------------------------------

--- 📝 Resultados do agente 4 (Revisor) ---



> O rascunho está ótimo, mas podemos fazer alguns ajustes para deixá-lo ainda mais engajador e alinhado com o público do Instagram:
> 
> *   **Emojis:** Moderar um pouco o uso de emojis pode dar um ar mais profissional e menos "infantil" ao post.
> *   **Linguagem:** Ajustar a linguagem para um tom mais conversacional e menos formal, sem perder a credibilidade.
> *   **Chamada para ação (CTA):** Tornar a CTA mais direta e convidativa.
> 
> **Sugestão de Rascunho Revisado:**
> 
> 🤖 Já pensou em ter um "faz tudo" da tecnologia trabalhando na sua empresa? Os agentes de IA autônomos estão chegando pra revolucionar a forma como os negócios funcionam! 🚀
> 
> ✨ O que eles fazem?
> Esses agentes são super inteligentes e conseguem:
> 🗓️ Agendar compromissos (sem dor de cabeça!)
> 📊 Analisar dados complexos (e te dar insights valiosos)
> 💬 Atender clientes de forma personalizada (e eficiente!)
> 
> 💼 Onde eles atuam?
> Na saúde, ajudam a monitorar pacientes e agendar consultas. Nas finanças, detectam fraudes e analisam o mercado. Na logística, otimizam rotas e gerenciam estoques. 🩺💰🚚
> 
> A Microsoft tá investindo pesado nessa tecnologia e a gente da Alura tá de olho em tudo! 👀
> 
> 🤔 E aí, como você acha que os agentes de IA podem turbinar o seu trabalho? Deixe sua opinião nos comentários! 👇
> 
> \#IA \#AgentesDeIA \#InteligenciaArtificial \#Microsoft #Alura #Inovação #Tecnologia
> 
> **Justificativas das mudanças:**
> 
> *   **Emojis:** Reduzi a quantidade para um visual mais limpo e profissional.
> *   **Linguagem:** Simplifiquei a linguagem, usando expressões como "sem dor de cabeça" e "turbinar o seu trabalho" para criar uma conexão mais próxima com o público.
> *   **CTA:** Reforcei a chamada para ação, incentivando o engajamento nos comentários de forma mais direta.
> *   **Hashtags:** Adicionei hashtags mais específicas e relevantes para aumentar o alcance do post.


---------------------------------------------
